#### Person Daily FOS Count

In [ ]:
import arcpy
from arcpy import na 
import arcpy.nax   
import os
import time

# Set environment
workspace = rf"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\MyPwoDSTPProject.gdb"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True 

input_dir = rf"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\FOS_Count"
out_dir = fr"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\Person_daily_FOS_Count"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Dictionary to group shapefiles by hhmemberid
shapefile_groups = {}

# Iterate through the input_dir to find all shapefiles
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.shp'):
            # Extract hhmemberid from each shapefile name
            hhmemberid = file.split('_')[2] + '_' + file.split('_')[3]

            # Add shapefiles to their respective groups in the dictionary based on hhmemberid
            if hhmemberid not in shapefile_groups:
                shapefile_groups[hhmemberid] = []
            shapefile_groups[hhmemberid].append(os.path.join(root, file))
# Print total number of hhmemberid in shapefile_groups
total_hhmemberid = len(shapefile_groups)
print(f"Total number of unique hhmemberid: {total_hhmemberid}")

# Iterate through the dictionary, and for each hhmemberid group:
for hhmemberid, shapefiles in shapefile_groups.items():
    # Create a new temporary shapefile to hold the merged geometries
    temp_shapefile_path = os.path.join(out_dir, f"temp_{hhmemberid}.shp") 
    # Use the Merge_management tool to merge the shapefiles in the group into the temporary shapefile
    arcpy.Merge_management(shapefiles, temp_shapefile_path)
    # Use the DeleteIdentical_management tool to remove duplicate points from the temporary shapefile
    arcpy.DeleteIdentical_management(temp_shapefile_path, ["SHAPE"])
    # Save the cleaned-up shapefile with the required naming convention to out_dir
    final_shapefile_name = f"person_{hhmemberid}_dailyFOS.shp"
    final_shapefile_path = os.path.join(out_dir, final_shapefile_name)
    # Check if the final_shapefile_path already exists
    if arcpy.Exists(final_shapefile_path):
        arcpy.Delete_management(final_shapefile_path)
    # Now, rename the temporary shapefile
    arcpy.Rename_management(temp_shapefile_path, final_shapefile_path)

#### MBG for person & Import Person FOS count and DPPA in csv file

In [ ]:
import arcpy
import os
import pandas as pd

# Set environment
workspace = rf"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\MyPwoDSTPProject.gdb"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

input_shp_file_directory = fr"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\Person_daily_FOS_Count"
MBG_directory = fr"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\MBG"
output_csv_dir = fr"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP"

# Create the MBG directory if it doesn't exist
if not os.path.exists(MBG_directory):
    os.makedirs(MBG_directory)

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=["hhmemberid", "FOSCount", "MBGArea"])

# List and process shapefiles for MBG
arcpy.env.workspace = input_shp_file_directory
shapefiles = [f for f in arcpy.ListFeatureClasses() if f.lower().endswith(".shp")]

for shapefile in shapefiles:
    hhmemberid = '_'.join(shapefile.split("_")[1:3])
    shapefile_path = os.path.join(input_shp_file_directory, shapefile)

    # Generate Minimum Bounding Geometry
    output_shapefile_name = os.path.splitext(shapefile)[0] + "_MBG.shp"
    output_shapefile_path = os.path.join(MBG_directory, output_shapefile_name)
    arcpy.MinimumBoundingGeometry_management(shapefile_path, output_shapefile_path, "CONVEX_HULL", "ALL")

    # Project the shapefile to desired projection (NAD 1983 UTM Zone 12N)
    projected_shapefile_path = output_shapefile_path.replace(".shp", "_proj.shp")
    arcpy.Project_management(output_shapefile_path, projected_shapefile_path, arcpy.SpatialReference(26912))

    # Calculate area and count
    with arcpy.da.SearchCursor(projected_shapefile_path, ["SHAPE@AREA"]) as cursor:
        total_area = sum([row[0] for row in cursor])
    feature_count = arcpy.GetCount_management(shapefile_path)[0]

    # Add data to DataFrame
    temp_df = pd.DataFrame({"hhmemberid": [hhmemberid], "FOSCount": [feature_count], "MBGArea": [total_area]})
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Save results to CSV
output_csv_path = os.path.join(output_csv_dir, "Resulted_FOS_DPPA.csv")
results_df.to_csv(output_csv_path, index=False)

print("CSV file has been saved successfully.")